In [1]:
# main_dir = "/Users/andrebeukers/Documents/fMRI/RVstudy/fromServ/Nov24_anatomicaFFA"
# main_dir = "/Users/andrebeukers/Documents/fMRI/RVstudy/fromServ/Nov2_RSA-FFApeakvox_mask"
# main_dir = "/Users/andrebeukers/Documents/fMRI/RVstudy/fromServ/Nov24_anatomical_visual"
# main_dir = "/Users/andrebeukers/Documents/fMRI/RVstudy/fromServ/Nov27_anatomicalFFA2"
main_dir = "/Users/andrebeukers/Documents/fMRI/RVstudy/fromServ/Nov29_frontalROIs"
mask_dir = "/Users/andrebeukers/Documents/fMRI/RVstudy/visser_replicate/BIDSdataset/masks"

In [2]:
# system
import os
from os.path import join as opj
from glob import glob

# data
import mvpa2.suite as mvpa
import numpy as np
import pandas as pd

# plots
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

In [3]:
task = 'pain'
roi = 'left_supfront'
meanCSp = 0

## Dirs

In [4]:
GLMresults_dir = opj(main_dir, roi, 'GLM' )
RSAresults_dir = opj(main_dir, roi, 'RSA')

if not os.path.isdir(RSAresults_dir):
    os.mkdir(RSAresults_dir)

In [5]:
# list of GLM result files
GLMresults_list = glob(opj(GLMresults_dir,"*%s*nifti*" % roi))

# mask file
mask_nii = opj(mask_dir, "%sROI.nii" % roi)

# behavioural ratin
ratings = pd.read_csv("/Users/andrebeukers/Documents/fMRI/RVstudy/visser_replicate/BIDSdataset/mana_rate.csv")

## Sample Attributes

In [6]:
t = lambda x,y: np.tile(x,y)
tn = lambda x: np.arange(x)+1

In [7]:
roi_list = t(roi, 51)
task_list = t(task, 51)

# assume order of stim_times_IM in GLM file unchanged
trialtype_list = np.concatenate(
    ( t('tp1',7), t('tp2',7), t('tm',7), t('cp',12), t('cm',6), t('us',12) ))

trialnum_list = np.concatenate((np.arange(7)+1,np.arange(7)+1,np.arange(7)+1,np.arange(30)))
stim_list = np.concatenate((t('CS+1',7),t('CS+2',7),t('CS-',7),t('0',30)))
CS_list = np.concatenate((t('CS+',14),t('CS-',7),t('0',30)))
trial_list = np.char.array(stim_list) + ' ' + np.char.array(trialnum_list)

# RSA loop

In [8]:
# prepare DSM 
dsm_measure = mvpa.measures.rsa.PDist(square=True)

# loop containers
ds_list = []; dsm_list = []

In [9]:
# loop beta volumes (GLM results), load fmri_dataset
for fpath in GLMresults_list:

    # file fpath and subject number
    sub_num = fpath.split('/')[-1].split('_')[0].split('-')[-1]
    print "RSAing subj: " + sub_num

    # conditioner score 
    condit = ratings[ratings['Subject'] == int(sub_num)]['Conditioner']
    
    # load beta volumes of single subject
    sub_ds = mvpa.fmri_dataset(fpath, mask=mask_nii)
#     assert sub_ds.shape == (51, 631)
    
    # sample attributes
    sub_ds.sa['sub_num'] = \
        (np.ones(sub_ds.nsamples)*int(sub_num)).astype(int)
    sub_ds.sa['trialtype'] = trialtype_list
    sub_ds.sa['trialnum'] = trialnum_list
    sub_ds.sa['roi'] = roi_list
    sub_ds.sa['task'] = task_list
    sub_ds.sa['stim'] = stim_list
    sub_ds.sa['trial'] = trial_list
    sub_ds.sa['condit'] = np.tile(condit,51)
    sub_ds.sa['CS'] = CS_list

    # get target trials only
    idx_tp1 = (sub_ds.sa.trialtype == 'tp1')
    idx_tp2 = (sub_ds.sa.trialtype == 'tp2')
    idx_tm = (sub_ds.sa.trialtype == 'tm')
    sub_ds = sub_ds[idx_tp1 | idx_tp2 | idx_tm]
#     assert sub_ds.shape == (21,631)

    # RDM (0,2) -> RSM (-1,1)
    sub_dsm = dsm_measure(sub_ds)
    sub_dsm.samples = sub_dsm.samples * (-1) + 1
    
    # fa
    sub_dsm.fa['trial'] = sub_dsm.sa.trial[:]

    # list of DSMs and DSs of each subject
    prob_subs = [4, 6, 19, 20, 335]
    if np.unique(sub_ds.sa.sub_num) not in prob_subs:
        dsm_list.append(sub_dsm)
        ds_list.append(sub_ds)


RSAing subj: 007
 * Please note: warnings are printed only once, but underlying problem might occur many times *
RSAing subj: 009
RSAing subj: 012
RSAing subj: 013
RSAing subj: 014
RSAing subj: 015
RSAing subj: 016
RSAing subj: 017
RSAing subj: 018
RSAing subj: 021
RSAing subj: 022
RSAing subj: 024
RSAing subj: 025
RSAing subj: 026
RSAing subj: 027
RSAing subj: 028
RSAing subj: 029
RSAing subj: 030
RSAing subj: 033
RSAing subj: 359


#### output: dsm_list

# Group analysis

In [10]:
full_dsm_stack = mvpa.vstack(dsm_list, a=0) 

### r-to-z

In [11]:
def fischer(ds_stack):
    
    # initialize dataset
    full_zstack = ds_stack.copy(deep=True)
    
    # apply arctanh
    full_zstack.samples = \
        np.arctanh(ds_stack.samples)
    
    return full_zstack

full_dsm_zstack = fischer(full_dsm_stack)


/Users/andrebeukers/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: RuntimeWarning: divide by zero encountered in arctanh


### split conditioners vs non

In [12]:
# conditioners 
condit_idx = full_dsm_zstack.sa.condit == 1
condit_dsm_zstack = full_dsm_zstack[condit_idx]

# non
noncondit_idx = full_dsm_zstack.sa.condit == 0
noncondit_dsm_zstack = full_dsm_zstack[noncondit_idx]

### average dsm

In [13]:
# measure
mgs = mvpa.mean_group_sample(["trial"], order='occurrence')

# avgRSM group, condit and non
dsm_c = mgs(condit_dsm_zstack)
dsm_n = mgs(noncondit_dsm_zstack)
dsm_group = mgs(full_dsm_zstack)

# for figure labeling
dsm_c.a['title'] = "conditioners"
dsm_n.a['title'] = "non-conditioners"
dsm_group.a['title'] = "group"

### z-to-r

In [14]:
dsm_c.samples = np.tanh(dsm_c.samples)
dsm_n.samples = np.tanh(dsm_n.samples)
dsm_group.samples = np.tanh(dsm_group.samples)

# note all > 0.1
for dsm in [dsm_c,dsm_n,dsm_group]:
    print dsm.samples.min()
    print dsm.samples.max()
    

-0.288537954538
1.0
-0.105099730948
1.0
-0.0785793634858
1.0


# Matrix Plots

In [15]:
def plot_dsm(dsm, title = 'Corr distance'):
    
    # figure
    fig = plt.figure(figsize=(10,8))
    plt.imshow(dsm, interpolation='nearest')
    
    # axes
    plt.xticks(np.arange(len(dsm))+.5, dsm.sa.trial, rotation=-45)
    plt.yticks(range(len(dsm)), dsm.fa.trial)
    plt.title(title)
    plt.clim(dsm.samples.min(), dsm.samples.max())
    plt.colorbar()

    # black square
    ax = fig.add_subplot(111)
    for xy in [-.5,6.5,13.5]:
        ax.add_patch(mpl.patches.Rectangle((xy,xy),
            7,7,linewidth=3,color='black',fill=False))

Note features: CS+1 07 

### Group RSM

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
# initialize figure saving
pp = PdfPages(opj(RSAresults_dir,'groupRSM_%s.pdf'%roi))

for dsm in [dsm_group, dsm_c, dsm_n]:
      
    # plot
    plot_dsm(dsm, title=dsm.a.title)
    pp.savefig()
    
pp.close()

### Subjects RSMs

In [ ]:
pp = PdfPages(opj(RSAresults_dir,'subjectRSM_%s.pdf'%roi))

for sub_dsm in dsm_list:
    
    plot_dsm(sub_dsm, title=np.unique(sub_dsm.sa.sub_num)[0])
    pp.savefig()
    
pp.close()


## Distributions

In [ ]:
ds30 = np.transpose(ds_list[-3].samples) # sub30

In [ ]:
t = 10

idx = ds30[:,t] != 0
samp = ds30[:,t][idx]

plt.hist(samp, bins=40)
plt.title("trial No. %i" % t)

In [ ]:
t = 7

idx = ds30[:,t] != 0
samp = ds30[:,t][idx]

plt.hist(samp, bins=40)
plt.title("trial No. %i" % t)

# Line plots

In [ ]:
dsm = dsm_list[0]


def get_diag(dsm):
    
    # off diagonal indices
    col_idx = np.arange(6) 
    row_idx = np.arange(6) + 1

    diag = {}
    for s in ['CS+1','CS+2','CS-']:

        # minor matrix with stim s only
        stim_idx = dsm.sa.stim == s
        stim_dsm = dsm[stim_idx,stim_idx]

        # off diag entries for s
        stim_diag = stim_dsm.samples[row_idx,col_idx]    
        diag[s] = stim_diag

    return diag

    
def plot_line(dsm, title = "trial-by-trial change"):
    plt.figure()
    
    # get diagonal entries
    diag_dict = get_diag(dsm)
    
    # plot CS+1 CS+2 CS-
    for stim, diag in diag_dict.iteritems():
        plt.plot(diag, label=stim)
        plt.title(title)
        plt.legend()
    
    # label x axis
    xlab = ["%i - %i" % (i, i+1) for i in range(8)[1:]]
    plt.xticks(np.arange(6), xlab, rotation = -45)
    

## group line plots

In [ ]:
pp = PdfPages(opj(RSAresults_dir,'group_lineplt.pdf'))

for dsm in [dsm_group, dsm_c, dsm_n]:
    
    plot_line(dsm, title="%s %s" % (dsm.a.title,roi))
    pp.savefig()
    
    
pp.close()

## subject line plots

In [ ]:
pp = PdfPages(opj(RSAresults_dir,'subject_lineplt.pdf'))

for sub_dsm in dsm_list:
    sub_num = np.unique(sub_dsm.sa.sub_num)[0]
    plot_line(sub_dsm, title="%i %s"% (sub_num,roi))
    pp.savefig()
    
    
pp.close()